# Comparing similarity measures

In this notebook a comparison between experimental results on humans and different similarity metrics are compared. 
This version uses dictionaries to avoid running the program many times.

# PENDING...

 - Normalizar todo.. 
 - Respaldar en github...
 - Hacer comparaciones contra individuos especificos de la encuesta (mi respuesta y otras que parezcan coherentes..)
   (PARA ESO debo de MARCAR las preguntas que eliminé...)
 - "Normalizar" datos, en vez de 1 - 6 pasar de 0 a 1... 
 - Hacer un conjunto de prueba hecho por mi, de 20 parejas
 - Comenzar a añadir 'la otra forma' de codificar... hacer algunas pruebas básicas... 
 - Viernes enseñar resultados a Barrón y preguntarle si añado lo de la otra forma de codificar... 


### Importing libraries

In [76]:
import pandas as pd
import nltk
import matplotlib
import matplotlib.pyplot as plt
import numpy
from scipy.stats.stats import pearsonr   

%matplotlib inline

#nltk.download('wordnet')
#nltk.download('wordnet_ic')
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic

exp_file = 'ExperimentalResults_2.xlsx'

### Getting pairs of concepts

In [154]:
def ListofPairs (number):
    "It obtains a list of pairs of concepts"
    df = pd.read_excel(exp_file)
    if number > 0:
        string = 'Q' + str(number)    
    else:
        string = 'average'
    # 1) List of concepts
    ordered = df.sort_values(by=string)
    c1 = map(str, list( ordered['concept 1'] ))
    c2 = map(str, list( ordered['concept 2'] ))
    L1 = map(list, zip(c1,c2))
    # 2) Human similarity
    ordered = df.sort_values(by=string)
    L2 = map(lambda x: round(float(x), 3) / 10, list(ordered[string]))
    
    return L1, L2

PConcepts, Hum_Sim = ListofPairs(0)
print PConcepts
#Hum_Sim = HumanSim()
# Plotting
#plt.plot(Hum_Sim)

[['spoon', 'barrel'], ['shoes', 'bike'], ['spoon', 'truck'], ['chair', 'pen'], ['spoon', 'envelope'], ['shoes', 'whip'], ['spoon', 'box'], ['shoes', 'shield'], ['chair', 'book'], ['table', 'brush'], ['spoon', 'basket'], ['table', 'curtains'], ['spoon', 'ashtray'], ['table', 'barrel'], ['chair', 'car'], ['table', 'lamp'], ['carpet', 'scarf'], ['shoes', 'carpet'], ['bed', 'lamp'], ['bed', 'mink_(coat)'], ['table', 'knife'], ['chair', 'closet'], ['shoes', 'earmuffs'], ['chair', 'dresser'], ['table', 'spoon'], ['bed', 'curtains'], ['shoes', 'skirt'], ['chair', 'bookcase'], ['shoes', 'mink_(coat)'], ['table', 'bookcase'], ['bowl', 'bathtub'], ['bed', 'chair'], ['bed', 'closet'], ['spoon', 'colander'], ['shoes', 'belt'], ['bed', 'dresser'], ['table', 'sofa'], ['bed', 'table'], ['boots', 'belt'], ['pen', 'envelope'], ['bed', 'pajamas'], ['stove', 'pot'], ['table', 'bench'], ['bed', 'cushion'], ['shovel', 'machete'], ['table', 'chair'], ['spoon', 'tongs'], ['bed', 'sofa'], ['spoon', 'spatula']

## HD Computing similarity

### Initializing memory and encoding dataset

In [19]:
%run KB_HDComputing.ipynb

# Initializing Memory
Init_mem()

End of initialization
End of encoding


### Comparing pairs of concepts in the HD binary space

To make computations more efficient I created a dictionary where the similarity between a pair of concepts is stored. Everytime that a similarity is required the only thing to do is to consult this dictionary

In [44]:
# Distance vector
HD_sim = []
for cc in PConcepts:
    HD_sim.append( HDvector.dist(Dict[cc[0]].getPointer(), Dict[cc[1]].getPointer()) )
#Normalizing...
HD_sim = map(lambda x: round(1. - x/float( max(HD_sim)) , 3), HD_sim)

# Dictionary of distances
Dict_HD = {}
for key in range(len(HD_sim)):
    Dict_HD[key] = HD_sim[key]

# Plotting
# plt.plot(HD_sim)

## McRae similarity

The following cells consult the similarity for each pair of concepts in the distance matrix provided.

In [47]:
def McRae_simi (pair_concepts):
    "Given a pair of concepts (in a list) it consults the similarity from the cos_matrix... file"
    try: 
        df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','1st_200')
        return list(df.loc[df['CONCEPT'] == pair_concepts[0]][pair_concepts[1]])[0]
    except:
        try:
            df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','2nd_200')
            return list(df.loc[df['CONCEPT'] == pair_concepts[0]][pair_concepts[1]])[0]
        except:
            df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','last_141')
            return list(df.loc[df['CONCEPT'] == pair_concepts[0]][pair_concepts[1]])[0]


# Dictionary of distances (McRae)
Dict_McRae = {}
key = 0
for cc in PConcepts:
    Dict_McRae[key] = McRae_simi(cc)
    key += 1

#plt.plot( )

## NTLK Library functions

### Auxiliary functions

In [48]:
brown_ic = wordnet_ic.ic('ic-brown.dat')

def get_synset (concept):
    "Given a concept name (string) it returns its synset (string)"
    # Dataframe for excel document
    df = pd.read_excel(pathh + 'CONCS_Synset_brm.xlsx') #../McRaedataset/CONCS_Synset_brm.xlsx')
    row = df.loc[df['Concept'] == concept]
    return str(list(row['Synset'] )[0])

def similarity_fun ( similarity_metric, pair, corpus = None):
    "Given a similarity_metric function it returns a list of the num closest concepts to 'concept'"
    c_synset_1 = wn.synset( get_synset(pair[0]))
    c_synset_2 = wn.synset( get_synset(pair[1]))
    if corpus:
        return round(similarity_metric(c_synset_1, c_synset_2, corpus), 3)
    else:
        return round(similarity_metric(c_synset_1, c_synset_2), 3)

## Similarity metrics

In [49]:
# Path similarity
Dict_path = {}
Dict_lch = {}
Dict_wup = {}
Dict_res = {}
Dict_jcn = {}
Dict_lin = {}

key = 0
for pair in PConcepts:
    Dict_path[key] = similarity_fun(wn.path_similarity, pair)
    Dict_lch[key] = similarity_fun(wn.lch_similarity, pair)
    Dict_wup[key] = similarity_fun(wn.wup_similarity, pair)
    Dict_res[key] = similarity_fun(wn.res_similarity, pair, brown_ic)
    Dict_jcn[key] = similarity_fun(wn.jcn_similarity, pair, brown_ic)
    Dict_lin[key] = similarity_fun(wn.lin_similarity, pair, brown_ic)
    key += 1
    
#plt.plot(Path_sim) #plt.plot(LC_sim) #plt.plot(WUP_sim) #plt.plot(Res_sim) #plt.plot(JC_sim) #plt.plot(Lin_sim)

## Correlations

Using correlation to compare obtained similarity values from different metrics.
Loop to obtain correlations between each questionary (and the average) and each similarity metric

In [185]:
Corr_mat = []
for Q in range(31):
    # 1) Obtain "human list", it can be the average responses or an specific one
    Hum_sim =  ListofPairs(Q) #or Q + #1-30
    
    # 2) Obtain list of keys for previous list
    keys = map(lambda x: PConcepts.index(x), Hum_sim[0])
    
    # 3) Create a list of distances according to each metric by consulting the appropiate dictionary
    HD_sim = [Dict_HD[x] for x in keys]
    McRae_sim = [Dict_McRae[x] for x in keys]
    path_sim = [Dict_path[x] for x in keys]
    lch_sim = [Dict_lch[x] for x in keys]
    # optional... 
    lch_sim = map(lambda x: round(x/float( max(lch_sim)) , 3), lch_sim)
    
    wup_sim = [Dict_wup[x] for x in keys]
    res_sim = [Dict_res[x] for x in keys]
    # optional
    res_sim = map(lambda x: round(x/float( max(res_sim)) , 3), res_sim)
    
    jcn_sim = [Dict_jcn[x] for x in keys]
    lin_sim = [Dict_lin[x] for x in keys]

    # 4) Calculate correlations
    correlations = [['HDC', pearsonr(Hum_sim[1], HD_sim)[0]], ['McRae', pearsonr(Hum_sim[1], McRae_sim)[0]],
                    ['path', pearsonr(Hum_sim[1], path_sim)[0]], ['lch', pearsonr(Hum_sim[1], lch_sim)[0]],
                    ['wup', pearsonr(Hum_sim[1], wup_sim)[0]], ['res', pearsonr(Hum_sim[1], res_sim)[0]],
                    ['jcn', pearsonr(Hum_sim[1], jcn_sim)[0]], ['lin', pearsonr(Hum_sim[1], lin_sim)[0]]]

    # 5) Sort list so far (key = )
    correlations = sorted(correlations, key = lambda x: x[1], reverse = True)
    
    # 6) Adding 'name' and average correlation value
    Corr_mat.append( ['Q' + str(Q), np.mean([r[1] for r in correlations])] + correlations )

# 7) Sorting entire matrix by average value (last...)
Corr_mat = sorted(Corr_mat, key = lambda x: x[-1], reverse = True)

# Printing
for v in Corr_mat:
    print v
    
# FALTA NORMALIZAR TODAS... 

['Q3', 0.5917556237420544, ['HDC', 0.6836631499222855], ['res', 0.6577614906127348], ['McRae', 0.6359506244359471], ['jcn', 0.6154477235373255], ['path', 0.5562118840318038], ['lin', 0.5491882602484144], ['lch', 0.5271583062509282], ['wup', 0.508663550896996]]
['Q24', 0.5361960095408826, ['HDC', 0.7475895935720432], ['McRae', 0.6536098717250649], ['jcn', 0.5565282522026359], ['res', 0.5326522166829488], ['path', 0.5124958597377782], ['lin', 0.4437791163246515], ['lch', 0.44339373603528853], ['wup', 0.39951943004664997]]
['Q25', 0.47116070895639545, ['HDC', 0.6927696532417578], ['McRae', 0.6399808913880194], ['jcn', 0.4581763074636498], ['res', 0.44663941701157794], ['lin', 0.4089437785521335], ['path', 0.4076076535268331], ['lch', 0.3707731502648828], ['wup', 0.34439482020230894]]
['Q30', 0.47116070895639545, ['HDC', 0.6927696532417578], ['McRae', 0.6399808913880194], ['jcn', 0.4581763074636498], ['res', 0.44663941701157794], ['lin', 0.4089437785521335], ['path', 0.4076076535268331], [

In [166]:
a = [0]
l = [1,2,3,4, 5]
print a.extend(l)
print a

None
[0, 1, 2, 3, 4, 5]
